# Accuracy

In this notebook, we will explore `accuracy` of the model. The metric commonly used with `loss` to `optimize` the model is `accuracy`. Accuracy describes how often the largest confidence is the correct class in terms of a fraction. ***The accuracy is calculated as the number of correct predictions divided by the total number of predictions.*** We will use the `argmax` values from the `softmax` outputs and then compare these to the targets. 

- We will calculate the accuracy of the model on the test data to see how well the model is performing. 
- We will also calculate the accuracy of the model on the training data to see if the model is overfitting or not. 
- We will also calculate the accuracy of the model on the validation data to see if the model is generalizing well or not. 
- We will also calculate the accuracy of the model on the unseen data to see if the model is generalizing well or not. 
- We will also calculate the accuracy of the model on the seen data to see if the model is generalizing well or not. 



In [1]:
import numpy as np

# Probabilities of 3 samples
softmax_outputs = np.array(
    [[0.7, 0.2, 0.1], [0.5, 0.1, 0.4], [0.02, 0.9, 0.08]]
)

# Target (ground-truth) labels for 3 samples
class_targets = np.array([0, 1, 1])

# TODO: We are also handling one-hot encoded targets by converting them to sparse values using np.argmax()
# Calculate values along second axis (axis of index 1)
predictions = np.argmax(softmax_outputs, axis=1)
# If targets are one-hot encoded - convert them
if len(class_targets.shape) == 2:
    class_targets = np.argmax(class_targets, axis=1)
# True evaluates to 1; False to 0
accuracy = np.mean(predictions == class_targets)
print("accuracy:", accuracy)

accuracy: 0.6666666666666666


## Accuracy of the Model

Now, we will calculate the accuracy of the model on the test data to see how well the model is performing. We will use a spiral dataset to test the model.

In [ ]:
# Now we create a dense layer with 3 neurons with 2 inputs each and 2 dense layers; the first layer has 3 neurons with 2 inputs each and the second layer has 3 neurons with 3 inputs each.
from src.layer.dense import Dense
from src.utils.datasets import create_spiral_dataset
from src.functions.activation import Softmax, ReLU
from scipy.ndimage import zoom

# Initialize activation function
softmax = Softmax()
relu = ReLU()

# Create a spiral dataset
X, y = create_spiral_dataset(100, 3)
y = np.array([y])
print(f"Inputs: {X.shape}")
print(f"Y is a spiral dataset: {y.shape}")

# Create a dense layer with 3 neurons with 2 inputs each
dense = Dense(2, 3)

# Lets do the forward pass
dense.forward(X)
print(f"Weights Layer 1: {dense.weights.shape}")
print(f"Biases Layer 1: {dense.biases.shape}")
print(f"Output Layer 1: {dense.output.shape}")

# Run the activation function ReLU
dense_output = relu(dense.output)

# Create a dense layer with 3 neurons with 3 inputs each
dense2 = Dense(3, 3)

# Lets do the forward pass
dense2.forward(dense_output)
print(f"Weights Layer 2: {dense2.weights.shape}")
print(f"Biases Layer 2: {dense2.biases.shape}")
print(f"Output Layer 2: {dense2.output.shape}")

# TODO: These final outputs are also our “confidence scores.” The higher the confidence score, the more confident the model is that the input belongs to that class.

# Run the activation function ReLU
predictions = softmax(dense2.output)
print(f"Pre Resize Predictions: {predictions.shape}")

# Match the size of predictions to the size of y
# Calculate zoom factor
zoom_factor = np.array(y.shape) / np.array(predictions.shape)
# Resize the predictions
predictions = zoom(
    predictions, zoom_factor, order=3
)  # 'order=3' for cubic interpolation
# predictions = np.array([predictions[range(len(predictions)), y[0]]])
print(f"Post Resize Predictions: {predictions.shape}")
print(f"True labels shape: {y.shape}")
print(f"Predictions Data[:1,0]: \n{predictions[:1,0]}")

# Calculate the loss and print the results to 7 decimal places
avg_loss, loss = dense2.loss(predictions, y)
print(f"Loss: {avg_loss:.7f}")  # Loss: 5.7037784
print(f"Loss Data Shape: \n{loss.shape}")

# Run ArgMax to get the predicted class
# Check the shape of the predictions
if len(predictions.shape) == 2:
    predictions_class = np.argmax(predictions, axis=0)
else:
    predictions_class = np.argmax(predictions, axis=1)

print(f"Predicted Class Shape: {predictions_class.shape}")
print(f"Predicted Class: {predictions_class[0]}")

# Calculate the accuracy
# Check the shape of the ground truth labels
# If targets are one-hot encoded - convert them
if len(y.shape) == 2:
    y = np.argmax(y, axis=0)
else:
    y = np.argmax(y, axis=1)
print(f"Ground Truth Shape: {y.shape}")
print(f"Ground Truth Class: {y[0]}")

# True evaluates to 1; False to 0
accuracy = np.mean(predictions_class == y)
print(f"Accuracy: {accuracy:.7f}")  # Accuracy: 0.3333333

## Adjusting Weights and Biases

Now, we will adjust the weights and biases of the model to improve the accuracy. We will use an iterative loop set to some large epoch value to adjust the weights and biases. We will also calculate the accuracy of the model on the test data to see how well the model is performing. We will use a vertical dataset to test the model.


In [ ]:
# Create a dense layer with 3 neurons with 2 inputs each and 2 dense layers; the first layer has 3 neurons with 2 inputs each and the second layer has 3 neurons with 3 inputs each.
from src.layer.dense import Dense
from src.functions.activation import Softmax, ReLU
from scipy.ndimage import zoom
from src.utils.datasets import create_vertical_data
import numpy as np
from src.utils.logger import getLogger

# Create a vertical dataset
X, y = create_vertical_data(100, 3)
y = y.reshape(-1, 1)
print(f"Inputs: {X.shape}")
print(f"Y is a vertical dataset: {y.shape}")

# Create a dense layer with 3 neurons with 2 inputs each
dense = Dense(2, 3)
print(f"Weights Layer 1: {dense.weights.shape}")
print(f"Biases Layer 1: {dense.biases.shape}")

# Create a dense layer with 3 neurons with 3 inputs each
dense2 = Dense(3, 3)
print(f"Weights Layer 2: {dense2.weights.shape}")
print(f"Biases Layer 2: {dense2.biases.shape}")

# Initialize activation function
softmax = Softmax()
relu = ReLU()

# We will create some variables to track the best loss, accuracy and the associated weights and biases
lowest_loss = 9999999
best_accuracy = 0
best_epoch = 0
best_weights = dense.weights.copy()
best_biases = dense.biases.copy()
best_weights2 = dense2.weights.copy()
best_biases2 = dense2.biases.copy()

# Set the learning rate
# learning_rate = 0.1

# Set the number of epochs
epochs = 10000
print(f"Epoch Count: {epochs}")
# Iterate over the epochs
for epoch in range(epochs):
    # Generate a new set of weights and biases for each iteration
    dense.weights = 0.5 * np.random.randn(2, 3)
    dense.biases = 0.1 * np.random.randn(1, 3)
    dense2.weights = 0.5 * np.random.randn(3, 3)
    dense2.biases = 0.1 * np.random.randn(1, 3)
    y_copy = y.copy()

    # Forward pass
    dense.forward(X)
    dense.activation = relu(dense.output)
    dense2.forward(dense.activation)
    predictions = softmax(dense2.output)

    # Reshape the ground truth labels if transformed during the forward pass
    if len(y_copy.shape) == 1:
        y_copy = np.array([y_copy])
    elif len(y_copy.shape) > 2:
        y_copy = np.array([y_copy]).reshape(-1, 1)

    # Match the size of predictions to the size of y
    if (y_copy.shape[1], y_copy.shape[0]) != predictions.shape or (
        y_copy.shape[0],
        y_copy.shape[1],
    ) != predictions.shape:
        zoom_factor = np.array(y_copy.shape) / np.array(predictions.shape)
        predictions = zoom(predictions, zoom_factor, order=3)

    # Calculate the loss
    avg_loss, loss = dense2.loss(predictions, y_copy)

    # One-hot encode the predictions
    if len(predictions.shape) >= 2:
        predictions_class = np.argmax(predictions, axis=1)
    elif len(predictions.shape) == 1:
        predictions_class = np.argmax(predictions, axis=0)
    else:
        predictions_class = np.argmax(predictions)

    # One-hot encode the ground truth labels
    if len(y_copy.shape) >= 2:
        y_copy = np.argmax(y_copy, axis=1)
    elif len(y_copy.shape) == 1:
        y_copy = np.argmax(y_copy, axis=0)
    else:
        y_copy = np.argmax(y_copy)

    # Calculate the accuracy
    accuracy = np.mean(predictions_class == y_copy)

    # Check if the loss is lower than the previous lowest loss
    # and if the accuracy is higher than the previous best accuracy
    if avg_loss < lowest_loss or accuracy > best_accuracy:
        print(f"Epoch: {epoch}, Loss: {avg_loss:.7f}, Accuracy: {accuracy:.7f}")
        best_epoch = epoch
        lowest_loss = avg_loss
        best_accuracy = accuracy
        best_weights = dense.weights.copy()
        best_biases = dense.biases.copy()
        best_weights2 = dense2.weights.copy()
        best_biases2 = dense2.biases.copy()

    # Print the best weights and biases when the epoch is complete
    if epoch == epochs - 1:
        print(f"Total Epoch Count: {epochs}")
        print(
            f"Best Epoch: {best_epoch}, Best Loss: {lowest_loss:.7f}, Best Accuracy: {best_accuracy:.7f}"
        )
        print(f"Best Weights Layer 1: \n{best_weights}")
        print(f"Best Biases Layer 1: \n{best_biases}")
        print(f"Best Weights Layer 2: \n{best_weights2}")
        print(f"Best Biases Layer 2: \n{best_biases2}")